In [27]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# pip3 install lxml
import openpyxl
#### sudz4 -> future feature -> try to use xlwings to write to excel

# load secrets
load_dotenv()

True

In [28]:
def get_historical_data(symbol, lookback_days=180):
    iex_key = os.getenv("IEX_API_KEY") 
    historical_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/chart/{lookback_days}d?token={iex_key}'

    historical_data = requests.get(historical_url).json()
    df = pd.DataFrame(historical_data)
    return df[['date', 'high', 'low', 'open','close']]

# Fetch historical data
ticker = 'PLTR' # Palantir Technologies Inc
#### sudz4 -> think about getting a list, can do list of SP500 or sector, pass the list to the function and get the data
historical_df = get_historical_data(ticker)

# Display the historical data
# print(historical_df)

# Convert the 'date' column to datetime objects
historical_df['date'] = pd.to_datetime(historical_df['date'])

# Sort the DataFrame by the 'date' column
historical_df = historical_df.sort_values(by='date')

In [29]:
# display historical data info
print()
print('Periods:', len(historical_df))
print('Period Start Date:', historical_df['date'].iloc[0])
print('Period End Date:', historical_df['date'].iloc[-1])
print()

# display historical data partial view
print('Historical Data START')
print(historical_df.head(5))
print()
print('Historical Data END')
print(historical_df.tail(5)) # last 5 rows


Periods: 124
Period Start Date: 2023-08-07 00:00:00
Period End Date: 2024-02-01 00:00:00

Historical Data START
        date   high    low   open  close
0 2023-08-07  18.39  17.01  18.25  17.99
1 2023-08-08  17.49  16.10  17.49  17.04
2 2023-08-09  16.92  15.10  16.76  15.25
3 2023-08-10  15.87  15.25  15.44  15.41
4 2023-08-11  15.58  14.91  15.05  15.41

Historical Data END
          date     high    low   open  close
119 2024-01-26  16.7500  16.34  16.40  16.35
120 2024-01-29  17.1100  16.43  16.45  17.09
121 2024-01-30  17.2400  16.66  17.04  16.74
122 2024-01-31  16.7600  16.07  16.41  16.09
123 2024-02-01  16.4868  16.03  16.21  16.33


In [30]:
# make pandas dataframe setting show all rows and columns
pd.set_option('display.max_rows', None)

#display historical data FULL chart view
# historical_df # uncomment to display full chart view

In [31]:
# Calculate Ichimoku components
high_9 = historical_df['high'].rolling(window=9).max()
low_9 = historical_df['low'].rolling(window=9).min()
historical_df['tenkan_sen'] = (high_9 + low_9) / 2

high_26 = historical_df['high'].rolling(window=26).max()
low_26 = historical_df['low'].rolling(window=26).min()
historical_df['kijun_sen'] = (high_26 + low_26) / 2

historical_df['senkou_span_a'] = ((historical_df['tenkan_sen'] + historical_df['kijun_sen']) / 2).shift(26)

high_52 = historical_df['high'].rolling(window=52).max()
low_52 = historical_df['low'].rolling(window=52).min()
historical_df['senkou_span_b'] = ((high_52 + low_52) / 2).shift(26)

historical_df['chikou_span'] = historical_df['close'].shift(-26)

# print updated df
historical_df

,date,high,low,open,close,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
0,2023-08-07,18.3900,17.0100,18.250,17.99,NaN,NaN,NaN,NaN,15.60
1,2023-08-08,17.4900,16.1000,17.490,17.04,NaN,NaN,NaN,NaN,15.83
2,2023-08-09,16.9200,15.1000,16.760,15.25,NaN,NaN,NaN,NaN,15.33
3,2023-08-10,15.8700,15.2500,15.440,15.41,NaN,NaN,NaN,NaN,15.46
4,2023-08-11,15.5800,14.9100,15.050,15.41,NaN,NaN,NaN,NaN,15.15
5,2023-08-14,15.7400,14.9500,15.200,15.72,NaN,NaN,NaN,NaN,14.74
6,2023-08-15,15.8700,15.2900,15.610,15.37,NaN,NaN,NaN,NaN,14.00
7,2023-08-16,15.7250,15.1200,15.210,15.45,NaN,NaN,NaN,NaN,14.13
8,2023-08-17,15.4373,14.0350,15.390,14.15,16.21250,NaN,NaN,NaN,14.22
9,2023-08-18,14.6000,13.6800,13.880,14.40,15.58500,NaN,NaN,NaN,13.96


In [32]:
# save dataframe to .xlsx file
# header is the row with the column names
pd.DataFrame(historical_df).to_excel('/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/ichimoku_cloud/ichimoku_data/PLTR_180d_ichimoku.xlsx', header=True, index=False)

# print success message
print('PLTR_180d_ichimoku.xlsx file saved successfully!')

PLTR_180d_ichimoku.xlsx file saved successfully!


In [33]:
# print column names
print(historical_df.columns.to_list())

['date', 'high', 'low', 'open', 'close', 'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b', 'chikou_span']
